# Test OpenAI Agents SDK
- basic usage
- use langfuse for prompt repository, evaluation, observability, user feedback
- run batches of prompt async against lists and dataframes
- implement a workflow to write a daily AI newsletter

In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import langfuse
from langfuse import get_client
from langfuse import Langfuse
from langfuse.openai import openai
# from langfuse.openai import AsyncOpenAI
import logfire
from llm import LangfuseClient

from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent  # methods to apply prompts async to large batches
from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent



In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")
print(f"LangFuse           {langfuse.version.__version__}")
print(f"Logfire            {logfire.__version__}")


OpenAI:            1.107.0
OpenAI Agents SDK  0.2.11
Pydantic           2.11.7
LangFuse           3.3.4
Logfire            4.7.0


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

12:20:39 | NewsletterAgent.newsletter_agent | INFO | Test info message
12:20:39 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
12:20:39 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

In [5]:
# Configure logfire instrumentation.
# OpenAI used logfire from Pydantic AI and we need this for langfuse to handle some traces sent to OpenAI by Agents SDK
logfire.configure(
    service_name="my_agent_service", 
    send_to_logfire=False,
    console=False,  # Disable console output
)
logging.getLogger("logfire").setLevel(logging.WARNING)
# Set logfire logger to WARNING level to reduce output
logfire.instrument_openai_agents()

# Langfuse Prompt Repo

In [ ]:
# initialize langfuse for observability
# git clone https://github.com/langfuse/langfuse.git
# cd langfuse
# go to localhost:3000
# set up an org, project, get API keys and put in .env

lf_client = get_client()
 
# Verify connection
if lf_client.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")# Get production prompts
prompt = lf_client.get_prompt("newsagent/headline_classifier")

# Get prompt from repository by label
# You can use as many labels as you'd like to identify different deployment targets
prompt = lf_client.get_prompt("newsagent/headline_classifier", label="production")
print(prompt.prompt, "\n")
prompt = lf_client.get_prompt("newsagent/headline_classifier", label="latest")
print(prompt.prompt, "\n")

# Get by version number, usually not recommended as it requires code changes to deploy new prompt versions
prompt = lf_client.get_prompt("newsagent/headline_classifier", version=1)
print(prompt.prompt, "\n")


In [ ]:
system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/headline_classifier")
print("system prompt")
print(system_prompt)
print() 

print("user prompt")
print(user_prompt)
print() 

print("model")
print(model)

# Basic usage
- Run a prompt using agents
- Sessions
- Route through Langfuse for observability
- Save logs
- View traces and evals


In [ ]:
# Get current `production` version of the prompt via raw langfuse client
system_prompt = lf_client.get_prompt("swallow/system")
 
# Insert variables into prompt template
# compiled_prompt = prompt.compile(criticlevel="expert", movie="Dune 2")

system_prompt.prompt


In [ ]:
# delete this local db to store agent sessions 
[os.remove(f) for f in glob.glob('swallow.db*') if os.path.exists(f)]


In [ ]:
# run a chat using the OpenAI Agent class with a session (and through langfuse for observability)

user_prompt = lf_client.get_prompt("swallow/user1").compile()
print(user_prompt)

openai_client = AsyncOpenAI()

# async def main():
agent = Agent(
    name="Assistant",
    instructions=system_prompt.prompt,
    model=OpenAIResponsesModel(model="gpt-4.1", openai_client=openai_client),
)

# 1) Create (or reuse) a session. Use a durable DB path if you want persistence.
session = SQLiteSession("test_swallow_chat", "swallow.db")

result = await Runner.run(agent, user_prompt, session=session)
display(Markdown(result.final_output))

# loop = asyncio.get_running_loop()
# await loop.create_task(main())


In [ ]:
# 3) Next turns — just keep reusing the same session
result = await Runner.run(agent, "explain how that number was measured / computed", session=session)

display(Markdown(result.final_output))

### View langfuse trace in the langfuse console

![langfuse_trace.png](langfuse_trace.png)


# More advanced usage
- Structured JSON outputs, enables validation and safe passing downstream over long pipelines
- Map prompts to larger data sets asynchronously (e.g. send parallel batches of 50)


In [ ]:
# Pydantic output class for classifying headlines - request returning values in this class
class ClassificationResult(BaseModel):
    """A single headline classification result"""
    input_str: str = Field(description="The original headline text")
    output: bool = Field(description="Whether the headline is AI-related")

class ClassificationResultList(BaseModel):
    """List of ClassificationResult for batch processing"""
    results_list: list[ClassificationResult] = Field(description="List of classification results")


In [ ]:
prompt_name = 'newsagent/headline_classifier'
lf_prompt = lf_client.get_prompt(prompt_name)
print(lf_prompt.prompt, end="\n")
print()

system_prompt = lf_prompt.prompt[0]['content']
print('system prompt\n', system_prompt, end="\n")
print()

user_prompt = lf_prompt.prompt[1]['content']
print('user prompt\n', user_prompt, end="\n")

config = lf_prompt.config if hasattr(lf_prompt, 'config') else {}
print ('config\n', config, end="\n")

model = config.get("model", 'gpt-5')
print('model\n', model, end="\n")


In [ ]:
# send single prompts via LLMAgent asking for ClassificationResult structured output
system_prompt, user_prompt, model = LangfuseClient().get_prompt("newsagent/headline_classifier")

classifier = LLMagent(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    output_type=ClassificationResult,
    model=model,
    verbose=True,
    logger=logger
)

test_headlines = [
    "AI Is Replacing Online Moderators, But It's Bad at the Job",
    "Baby Trapped in Refrigerator Eats Own Foot",
    "Machine Learning Breakthrough in Medical Diagnosis",
    "Local Restaurant Opens New Location",
    "ChatGPT Usage Soars in Educational Settings"
]

result = await classifier.run_prompt(input_str=test_headlines[0])
print(result)
result = await classifier.run_prompt(input_str=test_headlines[1])
print(result)



In [ ]:
# LLMAgent in llm.py has multiple ways to request stuff
# Suppose we have 1000 headlines in a dataframe and we want to apply a prompt to each one.
# Some stuff we might want
# - structured output, like ideally apply prompts to this column and put results in a new column
# - output validation, so llm doesn't e.g. transpose rows or skip rows
# - batching , don't send 1000 at once but don't send a single headline with a large prompt 1000 times
# - concurrency / async processing, send many batches at once (but maybe specify some max concurrency)
# - retry logic with exponential backoff
# LLMagent supports
# - run_prompt_dict to use kwargs
# - prompt_dict to send a dict return an object or list , might also just run_prompt(**mydict)
# - prompt_batch to map prompt to a list and return structured object
# - filter_dataframe, map prompt to a Pandas DataFrame and return a Series for assignment

# note different output type
classifier = LLMagent(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    output_type=ClassificationResultList,
    model=model,
    verbose=True,
    logger=logger
)

# Format headlines as a single string for batch processing
headlines_str = str(test_headlines)
result = await classifier.prompt_dict({'input_str': headlines_str})
print(f"Batch result: {result}")


In [ ]:
# make batches and send multiple in parallel
headlines_df = pd.read_csv("test_headlines.csv")
headlines_df


In [ ]:
# filter_dataframe

FILTER_SYSTEM_PROMPT, FILTER_USER_PROMPT, model = LangfuseClient().get_prompt("newsagent/filter_system_prompt")

# output class for classifying headlines
class ClassificationResultId(BaseModel):
    """A single headline classification result"""
    id: int = Field("The news item id")
    input_str: str = Field(description="The original headline title")
    output: bool = Field(description="Whether the headline title is AI-related")

class ClassificationResultIdList(BaseModel):
    """List of ClassificationResult for batch processing"""
    results_list: list[ClassificationResultId] = Field(description="List of classification results")


classifier = LLMagent(
    system_prompt=FILTER_SYSTEM_PROMPT,
    user_prompt=FILTER_USER_PROMPT,
    output_type=ClassificationResultIdList,  
    model=model,  # Use a valid model
    verbose=False,
    logger=logger
)

headlines_df['isAI'] = await classifier.filter_dataframe(headlines_df[["id", "title"]])

headlines_df



In [ ]:
display(headlines_df.loc[headlines_df['isAI']])
display(headlines_df.loc[~headlines_df['isAI']])


In [ ]:
# test various models for speed, accuracy, cost
# see costs under tracing
# http://localhost:3000/
# 'gpt-5-mini' 9.5¢' 16.8¢, 'gpt-4.1-mini 3.2¢' 
models = ['gpt-5-mini', 'gpt-5-nano', 'gpt-4.1', 'gpt-4.1-mini',]

# ground truth to compare
correct_df = pd.read_csv("headline_classifier_ground_truth.csv")

result_tuples = []

for m in models:
    print(f"Starting evaluation for {m}...")
    
    # Start timing
    start_time = time.time()
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
    
    with lf_client.start_as_current_span(name=f"batch_classification_{m}_{timestamp}") as span:
        classifier = LLMagent(system_prompt,
                              user_prompt,
                              ClassificationResultList,
                              m,
                              verbose=False)
    
        # Run classification with tracing
        classification_result = await classifier.prompt_batch(
            list(headlines_df['title'].to_list())
        )
        
        # Add span metadata
        span.update(
            input={"headlines_count": len(headlines_df)},
            metadata={"model": m}
        )
        
    # Calculate elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # analyze results
    result_df = pd.DataFrame([(z.input_str, z.output) 
                              for z in classification_result], 
                             columns=["input", "output"])
    
    
    # Merge with ground truth to compare results
    comparison_df = result_df.merge(correct_df, on='input', suffixes=('_predicted', '_correct'))
    
    # Calculate accuracy
    comparison_df['is_correct'] = comparison_df['output_predicted'] == comparison_df['output_correct']
    accuracy = comparison_df['is_correct'].mean()
    correct_count = comparison_df['is_correct'].sum()
    total_count = len(comparison_df)
    
    # Find differences
    differences_df = comparison_df[~comparison_df['is_correct']].copy()
    
    print(f"Completed {m} in {elapsed_time:.2f} seconds")
    print(f"Accuracy: {correct_count}/{total_count} = {accuracy:.3f} ({accuracy*100:.1f}%)")

    if len(differences_df) > 0:
        print(f"Found {len(differences_df)} incorrect predictions:")
        print("-" * 80)
        for idx, row in differences_df.iterrows():
            print(f"Input: {row['input']}")
            print(f"Predicted: {row['output_predicted']}")
            print(f"Correct:   {row['output_correct']}")
            print("-" * 40)
    else:
        print("🎉 Perfect accuracy! No incorrect predictions.")
    
    print()  # Empty line for readability
    
    # Create tuple with (model_name, df, elapsed_time, accuracy, differences_df)
    result_tuples.append((m, result_df, elapsed_time, accuracy, differences_df))

# Summary comparison
print("=" * 60)
print("SUMMARY COMPARISON")
print("=" * 60)
summary_data = []
for model_name, df, elapsed_time, accuracy, differences_df in result_tuples:
    rate = len(df) / elapsed_time if elapsed_time > 0 else 0
    summary_data.append({
        'Model': model_name,
        'Accuracy': f"{accuracy:.3f}",
        'Accuracy %': f"{accuracy*100:.1f}%", 
        'Correct': f"{int(accuracy * len(df))}/{len(df)}",
        'Time (s)': f"{elapsed_time:.2f}",
        'Rate (pred/s)': f"{rate:.1f}",
        'Errors': len(differences_df)
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Find most common errors across all models
print("\n" + "=" * 60)
print("MOST COMMON ERRORS ACROSS ALL MODELS")
print("=" * 60)
all_errors = []
for model_name, _, _, _, differences_df in result_tuples:
    for _, row in differences_df.iterrows():
        all_errors.append({
            'input': row['input'],
            'predicted': row['output_predicted'],
            'correct': row['output_correct'],
            'model': model_name
        })

if all_errors:
    error_df = pd.DataFrame(all_errors)
    error_counts = error_df.groupby('input').size().sort_values(ascending=False)
    
    print("Headlines that multiple models got wrong:")
    for headline, count in error_counts.head(10).items():
        if count > 1:  # Only show errors made by multiple models
            models_wrong = error_df[error_df['input'] == headline]['model'].tolist()
            predicted_values = error_df[error_df['input'] == headline]['predicted'].unique()
            correct_value = error_df[error_df['input'] == headline]['correct'].iloc[0]
            
            print(f"\n❌ Error in {count}/{len(models)} models: {', '.join(models_wrong)}")
            print(f"   Headline: {headline}")
            print(f"   Predicted: {predicted_values}")
            print(f"   Correct: {correct_value}")

lf_client.flush()



In [ ]:
# access results:
# 'gpt-5-mini' 9.5¢' gpt-4.1 16.8¢, 'gpt-4.1-mini 3.2¢' 
# note that you can get faster cheaper results with gpt-4.1 mini with good accuracy
for model_name, df, elapsed_time, accuracy, error_df in result_tuples:
    print(f"{model_name}: {len(df)} results in {elapsed_time:.2f}s, accuracy {accuracy:.3f}")

# Run Agent Worfklow

# test agent



In [7]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# set up state
state = NewsletterAgentState() 

# Create agent with persistent state
timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")
session_id = f"test_newsletter_{timestamp}"
agent = NewsletterAgent(session_id=session_id, state=state, verbose=True)

# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


12:21:19 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Using provided state with 0 articles


🚀 Creating NewsletterAgent...
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20250919122119302709

📝 User prompt: 'Show the workflow status'


12:21:22 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Starting check_workflow_status
12:21:22 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Completed check_workflow_status


⏱️  Total execution time: 6.94s
📊 Final result:
Current workflow status:

- Progress: 0.0% (0/9 complete)
- Summary: 0 complete, 0 started, 0 failed, 9 not started
- Next step: Step 1 — Fetch URLs

Step-by-step status:
1. Fetch Urls: not_started
2. Filter Urls: not_started
3. Download Articles: not_started
4. Extract Summaries: not_started
5. Cluster By Topic: not_started
6. Rate Articles: not_started
7. Select Sections: not_started
8. Draft Sections: not_started
9. Finalize Newsletter: not_started

What would you like me to do next? Options: run all steps, run a specific step, or resume (start from Step 1).


In [8]:
# User prompt to run workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



📝 User prompt: 'Run step 1, fetch urls'


12:21:39 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Starting check_workflow_status
12:21:39 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Completed check_workflow_status
12:21:41 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Starting Step 1: Gather URLs
2025-09-19 12:21:41,741 - fetcher_4881505488 - INFO - [fetcher_init] Loading sources from sources.yaml
2025-09-19 12:21:41,757 - fetcher_4881505488 - INFO - [fetcher_init] Loaded 17 sources: 7 RSS, 9 HTML, 1 API
2025-09-19 12:21:41,758 - fetcher_4881505488 - DEBUG - [fetcher_sources] Source 'Ars Technica': type=RSS, url=https://arstechnica.com/ai/
2025-09-19 12:21:41,758 - fetcher_4881505488 - DEBUG - [fetcher_sources] Source 'Bloomberg': type=RSS, url=https://www.bloomberg.com/ai
2025-09-19 12:21:41,758 - fetcher_4881505488 - DEBUG - [fetcher_sources] Source 'Business Insider': type=html, url=https://www.businessinsider.com/tech
2025-09-19 12:21:41,758 - fetcher_4881505488 - DEBU

2025-09-19 12:21:43,488 - fetcher_4881505488 - INFO - Starting scrape_source https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day, Reddit
2025-09-19 12:21:43,488 - fetcher_4881505488 - INFO - scrape_url(https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+tech+technews+technology/top/?sort=top&t=day)
2025-09-19 12:21:43,488 - fetcher_4881505488 - INFO - scraping https://www.reddit.com/r/AI_Agents+ArtificialInteligence+Automate+ChatGPT+ChatGPTCoding+Futurology+MachineLearning+OpenAI+ProgrammerHumor+accelerate+aiArt+aivideo+artificial+deeplearning+learnmachinelearning+programming+singularity+

2025-09-19 12:21:43,848 - fetcher_4881505488 - INFO - scraping https://www.theverge.com/ai-artificial-intelligence to download/sources
2025-09-19 12:21:43,848 - fetcher_4881505488 - INFO - File already exists: download/sources/The_Verge.html
2025-09-19 12:21:43,848 - fetcher_4881505488 - INFO - [fetch_html] Parsing HTML file: download/sources/The_Verge.html
2025-09-19 12:21:43,883 - fetcher_4881505488 - INFO - [fetch_html] Parsed HTML file: download/sources/The_Verge.html
2025-09-19 12:21:43,883 - fetcher_4881505488 - INFO - [fetch_html] HTML fetch successful for The Verge: 27 articles
2025-09-19 12:21:43,883 - fetcher_4881505488 - INFO - [fetch_html] Source dict for VentureBeat: {'type': 'html', 'url': 'https://venturebeat.com/category/ai/', 'filename': 'VentureBeat', 'rss': 'https://venturebeat.com/category/ai/feed/'}
2025-09-19 12:21:43,884 - fetcher_4881505488 - INFO - Starting scrape_source https://venturebeat.com/category/ai/, VentureBeat
2025-09-19 12:21:43,884 - fetcher_4881505

,source,url
0,Ars Technica,20
1,Bloomberg,33
2,Business Insider,17
3,FT,103
4,Feedly AI,71
5,Hacker News,30
6,HackerNoon,50
7,New York Times,28
8,NewsAPI,95
9,Reddit,57


✅ Completed Step 1: Gathered 692 URLs from 17 RSS sources


,source,title,url,published,rss_summary,id
0,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,0
1,Ars Technica,Google announces massive expansion of AI featu...,https://arstechnica.com/google/2025/09/google-...,"Thu, 18 Sep 2025 19:27:58 +0000",Chrome's future as an AI browser starts today.,1
2,Ars Technica,New attack on ChatGPT research agent pilfers s...,https://arstechnica.com/information-technology...,"Thu, 18 Sep 2025 16:29:22 +0000","Unlike most prompt injections, ShadowLeak exec...",2
3,Ars Technica,White House officials reportedly frustrated by...,https://arstechnica.com/ai/2025/09/white-house...,"Wed, 17 Sep 2025 22:03:11 +0000",Officials say Claude chatbot usage policies bl...,3
4,Ars Technica,Gemini AI solves coding problem that stumped 1...,https://arstechnica.com/google/2025/09/google-...,"Wed, 17 Sep 2025 17:00:32 +0000",Gemini shows off at another high-level academi...,4
...,...,...,...,...,...,...
687,NewsAPI,Sylvester Stallone wanted to use AI to “rewrit...,https://www.joblo.com/stallone-rambo-prequel-ai/,2025-09-18T14:07:27Z,NaN,687
688,NewsAPI,"Amazon adds Walmart, Shopify, Shein to Multi-C...",https://www.supplychaindive.com/news/amazon-wa...,2025-09-18T15:23:00Z,NaN,688
689,NewsAPI,Get Efficient: How William Wong is Bringing AI...,https://www.ibtimes.com/get-efficient-how-will...,2025-09-18T15:49:26Z,NaN,689
690,NewsAPI,Billionaire Ross Lures ServiceNow for South Fl...,https://www.insurancejournal.com/news/southeas...,2025-09-18T14:18:31Z,NaN,690


12:21:43 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Completed Step 1: Gathered 692 articles
12:21:45 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Starting check_workflow_status
12:21:45 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Completed check_workflow_status


⏱️  Total execution time: 11.63s
📊 Final result:
Step 1 (Fetch URLs) completed.

Status after running Step 1:
- Progress: 11.1% (1/9 complete)
- 692 articles gathered from 17 sources (RSS only)
- Next step: Step 2 — Filter URLs

Would you like me to continue to Step 2 (filter AI-related URLs) now?


In [9]:
state 


NewsletterAgentState(headline_data=[{'source': 'Ars Technica', 'title': 'AI medical tools found to downplay symptoms of women, ethnic minorities', 'url': 'https://arstechnica.com/health/2025/09/ai-medical-tools-found-to-downplay-symptoms-of-women-ethnic-minorities/', 'published': 'Fri, 19 Sep 2025 13:30:27 +0000', 'rss_summary': 'Bias-reflecting LLMs lead to inferior medical advice for female, Black, and Asian patients.', 'id': 0}, {'source': 'Ars Technica', 'title': 'Google announces massive expansion of AI features in Chrome', 'url': 'https://arstechnica.com/google/2025/09/google-announces-massive-expansion-of-ai-features-in-chrome/', 'published': 'Thu, 18 Sep 2025 19:27:58 +0000', 'rss_summary': "Chrome's future as an AI browser starts today.", 'id': 1}, {'source': 'Ars Technica', 'title': 'New attack on ChatGPT research agent pilfers secrets from Gmail inboxes', 'url': 'https://arstechnica.com/information-technology/2025/09/new-attack-on-chatgpt-research-agent-pilfers-secrets-from-

In [10]:
pd.DataFrame(state.headline_data)



,source,title,url,published,rss_summary,id
0,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,0
1,Ars Technica,Google announces massive expansion of AI featu...,https://arstechnica.com/google/2025/09/google-...,"Thu, 18 Sep 2025 19:27:58 +0000",Chrome's future as an AI browser starts today.,1
2,Ars Technica,New attack on ChatGPT research agent pilfers s...,https://arstechnica.com/information-technology...,"Thu, 18 Sep 2025 16:29:22 +0000","Unlike most prompt injections, ShadowLeak exec...",2
3,Ars Technica,White House officials reportedly frustrated by...,https://arstechnica.com/ai/2025/09/white-house...,"Wed, 17 Sep 2025 22:03:11 +0000",Officials say Claude chatbot usage policies bl...,3
4,Ars Technica,Gemini AI solves coding problem that stumped 1...,https://arstechnica.com/google/2025/09/google-...,"Wed, 17 Sep 2025 17:00:32 +0000",Gemini shows off at another high-level academi...,4
...,...,...,...,...,...,...
687,NewsAPI,Sylvester Stallone wanted to use AI to “rewrit...,https://www.joblo.com/stallone-rambo-prequel-ai/,2025-09-18T14:07:27Z,NaN,687
688,NewsAPI,"Amazon adds Walmart, Shopify, Shein to Multi-C...",https://www.supplychaindive.com/news/amazon-wa...,2025-09-18T15:23:00Z,NaN,688
689,NewsAPI,Get Efficient: How William Wong is Bringing AI...,https://www.ibtimes.com/get-efficient-how-will...,2025-09-18T15:49:26Z,NaN,689
690,NewsAPI,Billionaire Ross Lures ServiceNow for South Fl...,https://www.insurancejournal.com/news/southeas...,2025-09-18T14:18:31Z,NaN,690


In [11]:
# Run tool directly
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


12:23:21 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Starting Step 2: Filter URLs


🔍 Classifying 692 headlines using LLM...


,source,title,url,published,rss_summary,id
0,Ars Technica,AI medical tools found to downplay symptoms of...,https://arstechnica.com/health/2025/09/ai-medi...,"Fri, 19 Sep 2025 13:30:27 +0000",Bias-reflecting LLMs lead to inferior medical ...,0
1,Ars Technica,Google announces massive expansion of AI featu...,https://arstechnica.com/google/2025/09/google-...,"Thu, 18 Sep 2025 19:27:58 +0000",Chrome's future as an AI browser starts today.,1
2,Ars Technica,New attack on ChatGPT research agent pilfers s...,https://arstechnica.com/information-technology...,"Thu, 18 Sep 2025 16:29:22 +0000","Unlike most prompt injections, ShadowLeak exec...",2
3,Ars Technica,White House officials reportedly frustrated by...,https://arstechnica.com/ai/2025/09/white-house...,"Wed, 17 Sep 2025 22:03:11 +0000",Officials say Claude chatbot usage policies bl...,3
4,Ars Technica,Gemini AI solves coding problem that stumped 1...,https://arstechnica.com/google/2025/09/google-...,"Wed, 17 Sep 2025 17:00:32 +0000",Gemini shows off at another high-level academi...,4
...,...,...,...,...,...,...
687,NewsAPI,Sylvester Stallone wanted to use AI to “rewrit...,https://www.joblo.com/stallone-rambo-prequel-ai/,2025-09-18T14:07:27Z,NaN,687
688,NewsAPI,"Amazon adds Walmart, Shopify, Shein to Multi-C...",https://www.supplychaindive.com/news/amazon-wa...,2025-09-18T15:23:00Z,NaN,688
689,NewsAPI,Get Efficient: How William Wong is Bringing AI...,https://www.ibtimes.com/get-efficient-how-will...,2025-09-18T15:49:26Z,NaN,689
690,NewsAPI,Billionaire Ross Lures ServiceNow for South Fl...,https://www.insurancejournal.com/news/southeas...,2025-09-18T14:18:31Z,NaN,690


12:23:21 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Initialized LLMagent:
system_prompt: 
You are a content-classification assistant that labels news headlines as AI-related or not.
Return **only** a JSON object that satisfies the provided schema.
For each headline provided, you must return an element with the same id, and a boolean value; do not skip any items.
No markdown, no markdown fences, no extra keys, no comments.

user_prompt: 
Classify every headline below.

AI-related if the title mentions (explicitly or implicitly):
- Core AI technologies: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: ChatGPT, Gemini, Claude, Sora, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existi

12:23:21 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | User message: 
Classify every headline below.

AI-related if the title mentions (explicitly or implicitly):
- Core AI technologies: machine learning, neural / deep / transformer networks
- AI Applications: computer vision, NLP, robotics, autonomous driving, generative media
- AI hardware, GPU chip supply, AI data centers and infrastructure
- Companies or labs known for AI: OpenAI, DeepMind, Anthropic, xAI, NVIDIA, etc.
- AI models & products: ChatGPT, Gemini, Claude, Sora, Midjourney, DeepSeek, etc.
- New AI products and AI integration into existing products/services
- AI policy / ethics / safety / regulation / analysis
- Research results related to AI
- AI industry figures (Sam Altman, Demis Hassabis, etc.)
- AI market and business developments, funding rounds, partnerships centered on AI
- Any other news with a significant AI component

Non-AI examples: crypto, ordinary software, non-AI gadgets and medical devic

...

12:23:51 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Result: RunResult:
- Last agent: Agent(name="LLMagent", ...)
- Final output (AIClassificationList):
    {
      "results_list": [
        {
          "id": 525,
          "input_str": "Alphabet’s Google will invest $6.78 billion in the U.K. over the next two years, the latest big-ticket commitment from an American tech giant seeking to expand artificial-intelligence infrastructure and other services in Europe.",
          "output": true
        },
        {
          "id": 526,
          "input_str": "Casio Wants to Break Into Wellness Tech. Its Plan? A Fluffy AI Robot",
          "output": true
        },
        {
          "id": 527,
          "input_str": "Casio Computer is branching out with an AI-powered pet as rising loneliness makes more people turn to chatbots for companionship.",
          "output": true
        },
        {
          "id": 528,
          "input_str": "iOS 26: Liquid Glass Is Here and Yo

12:23:54 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Result: RunResult:
- Last agent: Agent(name="LLMagent", ...)
- Final output (AIClassificationList):
    {
      "results_list": [
        {
          "id": 75,
          "input_str": "Trump and the limits to British flattery",
          "output": false
        },
        {
          "id": 76,
          "input_str": "Trump fails to oust Lisa Cook from Fed ahead of rates decision",
          "output": false
        },
        {
          "id": 77,
          "input_str": "Trump files $15bn defamation lawsuit against New York Times",
          "output": false
        },
        {
          "id": 78,
          "input_str": "Judge dismisses state terrorism charges against Luigi Mangione",
          "output": false
        },
        {
          "id": 79,
          "input_str": "Beijing says TikTok’s US app will use Chinese algorithm",
          "output": true
        },
        {
          "id": 80,
          "input_str

12:23:57 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Result: RunResult:
- Last agent: Agent(name="LLMagent", ...)
- Final output (AIClassificationList):
    {
      "results_list": [
        {
          "id": 450,
          "input_str": "SK Hynix cranks up the HBM4 assembly line to prep for next-gen GPUs",
          "output": true
        },
        {
          "id": 451,
          "input_str": "Your call is very important to us – which is why we're connecting you to a human",
          "output": false
        },
        {
          "id": 452,
          "input_str": "Google lands £400M MoD contract for secure UK cloud services",
          "output": false
        },
        {
          "id": 453,
          "input_str": "Terminators: AI-driven robot war machines on the march",
          "output": true
        },
        {
          "id": 454,
          "input_str": "Albania’s prime minister wants to appoint an AI to his ministry",
          "output": true
        },
 

12:24:02 | NewsletterAgent.test_newsletter_20250919122119302709 | INFO | Result: RunResult:
- Last agent: Agent(name="LLMagent", ...)
- Final output (AIClassificationList):
    {
      "results_list": [
        {
          "id": 0,
          "input_str": "AI medical tools found to downplay symptoms of women, ethnic minorities",
          "output": true
        },
        {
          "id": 1,
          "input_str": "Google announces massive expansion of AI features in Chrome",
          "output": true
        },
        {
          "id": 2,
          "input_str": "New attack on ChatGPT research agent pilfers secrets from Gmail inboxes",
          "output": true
        },
        {
          "id": 3,
          "input_str": "White House officials reportedly frustrated by Anthropic\u0019s law enforcement AI limits",
          "output": true
        },
        {
          "id": 4,
          "input_str": "Gemini AI solves coding problem that stumped 139 human teams at ICPC World Finals",
  

✅ Completed Step 2: Filtered to 368 AI-related headlines from 692 total
⏱️  Total execution time: 41.50s
📊 Final result:
✅ Step 2 completed successfully! Filtered 692 headlines to 368 AI-related articles.

📊 Results stored in persistent state. Current step: step_01_fetch_urls


In [ ]:
# User prompt to run workflow
user_prompt = "Run step 3, download full articles"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
# User prompt to run workflow
user_prompt = "Run step 4, Summarize articles"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
state = await agent.run_step("get state")
state 


In [ ]:
state = await agent.get_state_direct()


In [ ]:
agent = NewsletterAgent(session_id="test_newsletter_20250918142308630453", verbose=True)
status_result = await agent.run_step("check workflow status")


In [ ]:
inspect_result = await agent.run_step("inspect state")


In [ ]:
print(status_result)


In [ ]:
# todos
# be less verbose
# show current headlines 
# try to call detailed state inspection tool from a prompt , showing count by source 
# get prompts from langfuse

In [ ]:
# class to store agent state from step to step
from newsletter_state import NewsletterAgentState


In [ ]:
import sys 

# if 'fetch' in sys.modules:
#     del sys.modules['fetch']
#     # Delete the reference
#     del Fetcher
from fetch import Fetcher

# should probably do this in the initialization based on parameters --nofetch
destination = "download/sources/"
for file in os.listdir(destination):
    file_path = os.path.join(destination, file)
    if os.path.isfile(file_path):
        os.remove(file_path)
        logger.info(f"Removed existing file: {file_path}")

In [ ]:
async with Fetcher() as f:
     z = await f.fetch_all()
z 


In [ ]:
len(z)

In [ ]:
for src in z:
    print(src['source'])
    print(src['status'])
    print(src['metadata'])
    print(len(src['results']))
          

In [ ]:
z[0]

In [ ]:
sources_results = z
successful_sources = []
failed_sources = []
all_articles = []

for result in sources_results:
    if result['status'] == 'success' and result['results']:
        # Add source info to each article
        successful_sources.append(result['source'])
        all_articles.extend(result['results'])
    else:
        failed_sources.append(result['source'])
        
headline_data = all_articles

In [ ]:
# TODO
len(all_articles)
headline_df = pd.DataFrame(all_articles)
display(headline_df[["source", "url"]].groupby("source") \
    .count() \
    .reset_index() \
    .rename({'url': 'count'}))


In [ ]:
f.sources.get('Ars Technica')


In [ ]:
from news_agent import NewsletterAgent


In [ ]:
news_agent = NewsletterAgent(session_id=f"newsletter_{random.randint(10000000, 99999999)}", verbose=True)


In [ ]:
user_prompt = "Can you show me the current status of the newsletter workflow"

start_time = time.time()
result = await news_agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)



In [ ]:
result


In [ ]:
# Create mock context
class MockContext:
    def __init__(self):
        self.context = news_agent.default_state

ctx = MockContext()
current_state = ctx.context  # From your previous run, or reload it
df = current_state.headline_df
df



In [ ]:
try:
    current_state = news_agent.session.get_state()
except:
    current_state = news_agent.default_state

print(current_state)
print()

print(f"Current Step: {current_state.current_step}/9")
print(f"Workflow Complete: {current_state.workflow_complete}")
print(f"Progress: {(current_state.current_step/9)*100:.1f}%")
print(f"Total articles: {len(current_state.headline_data)}")

if current_state.headline_data:
    ai_related = sum(1 for a in current_state.headline_data if a.get('ai_related') is True)
    print(f"AI-related articles: {ai_related}")
    print(f"Summaries: {len(current_state.article_summaries)}")
    print(f"Clusters: {len(current_state.topic_clusters)}")
    print(f"Sections: {len(current_state.newsletter_sections)}")

In [ ]:
# review slides

# review workflow status, move to a moadule
# all prints should be logs
# section writing and composition will have the critic /optimizer loop
# add batch with async


In [ ]:
def create_news_dataframe():
    """
    Creates an empty DataFrame to support headline/article analysis
    - URLs, source tracking and metadata
    - Topic classification and clustering
    - Content quality ratings and rankings

    Returns:
        pd.DataFrame: Empty DataFrame with predefined column structure
    """

    # column structure
    column_dict = {
        # Core identifiers and source info
        'article_id': 'object',              # Unique identifier for each article
        'source':     'object',              # Source category
        'headline_title': 'object',          # Article headline/title
        'original_url': 'object',            # Initial URL before redirects
        'final_url': 'object',               # URL after following redirects
        'domain_name': 'category',           # Website domain
        'site_name': 'category',             # Human-readable site name
        'site_reputation_score': 'float32',  # Reputation/trustworthiness score for the site
        'keep_flag': 'boolean',

        # File paths and storage
        'html_file_path': 'object',          # Path to stored HTML content
        'text_file_path': 'object',          # Path to extracted text content

        # Time information
        'last_updated_timestamp': 'datetime64[ns]',  # When article was last updated
        'article_age_days': 'int32',         # Age of article in days
        'recency_score': 'float32',          # Calculated recency score (higher = more recent)

        # Content analysis
        'content_summary': 'object',         # Generated summary of article content
        'bullet_points': 'object',           # Key points extracted as bullets
        'article_length_chars': 'int32',     # Character count of article content

        # Rating flags (LLM-generated probabilities)
        'is_high_quality': 'float32',        # LLM probability for low-quality content
        'is_off_topic': 'float32',           # LLM probability for off-topic content
        'is_low_importance': 'float32',      # 1-LLM probability for high-importance content

        # Other ratings
        'bradley_terry_score': 'float32',    # Bradley-Terry rating from pairwise article comparisons
        'bradley_terry_rank': 'int32',       # Ordinal rank based on Bradley-Terry scores (1 = highest rated)
        'adjusted_length_score': 'float32',  # Length-adjusted quality score
        'final_composite_rating': 'float32', # Final weighted rating combining multiple factors

        # Topic classification
        'topic_string': 'object',            # Topic labels as comma-separated string
        'topic_list': 'object',              # Topic labels as list/array structure (same topics, different format)

        # Organization and clustering (HDBSCAN-based)
        'display_order': 'int32',            # Order for display/presentation
        'cluster_id': 'int32',               # HDBSCAN cluster identifier (-1 = noise/outlier)
        'cluster_label': 'category'          # Human-readable cluster name/description
    }

    # Create empty DataFrame from column dictionary
    df = pd.DataFrame(columns=list(column_dict.keys())).astype(column_dict)

    return df



In [ ]:
@dataclass
class NewsletterState:
    """
    Maintains session state for the OpenAI Agents SDK workflow.

    Attributes:
        headline_df: DataFrame containing headline data for processing
        sources_file: Path to YAML file containing source configurations
        sources: Dictionary of source configurations loaded from YAML
        cluster_topics: List of clean topic names for headline categorization
        max_edits: Maximum number of critic optimizer editing iterations allowed
        edit_complete: Boolean flag indicating if editing process is finished
        n_browsers: Number of concurrent Playwright browser instances for downloads
    """

    status: WorkflowStatus = WorkflowStatus()
    headline_df: pd.DataFrame = field(default_factory=create_news_dataframe)
    sources_file: str = field(default="sources.yaml")
    sources: Dict[str, Any] = field(default_factory=dict)
    cluster_topics: List[str] = field(default_factory=list)
    max_edits: int = field(default=3)
    edit_complete: bool = field(default=False)
    n_browsers: int = field(default=8)
    verbose: bool = field(default=True)


    def __post_init__(self):
        """
        Post-initialization validation and setup.

        Validates that the configuration makes sense and performs
        any necessary initialization steps.
        """
        # Validate max_edits is reasonable
        if self.max_edits < 1 or self.max_edits > 10:
            raise ValueError(f"max_edits should be between 1-10, got {self.max_edits}")

        # Validate n_browsers is reasonable
        if self.n_browsers < 1 or self.n_browsers > 32:
            raise ValueError(f"n_browsers should be between 1-32, got {self.n_browsers}")

        # Validate sources_file exists and load sources from file automatically
        try:
            sources_path = Path(self.sources_file)
            with open(sources_path, 'r', encoding='utf-8') as file:
                self.sources = yaml.safe_load(file) or {}
            if self.verbose:
                print(f"Loaded {len(self.sources)} sources from {self.sources_file}")
        except FileNotFoundError:
            raise FileNotFoundError(f"Sources file not found: {self.sources_file}")
        except yaml.YAMLError as e:
            raise ValueError(f"Error parsing YAML file {self.sources_file}: {e}")


In [ ]:
state = NewsletterState()
state


In [ ]:
from agents import Agent, Runner, SQLiteSession, function_tool, RunContextWrapper


In [ ]:
class NewsletterAgent(Agent[NewsletterState]):
    """AI newsletter writing agent with structured workflow"""

    def __init__(self, session_id: str = "newsletter_agent"):
        self.session = SQLiteSession(session_id, "newsletter.db")
        self.state = NewsletterState()

        super().__init__(
            name="AINewsletterAgent",
            instructions="""
            You are an AI newsletter writing agent. Your role is to:
            1. Scrape headlines and URLs from various sources
            2. Filter the headlines to ones that are about AI
            3. Fetch the URLs and save them as plain text
            4. Summarize each article to 3 bullet points containing the key facts
            5. Extract topics from each article and cluster articles by topic
            6. Rate each article according to the provided rubric
            7. Identify 6-15 thematic sections + "Other News", assign articles to sections and deduplicate
            8. Write each section
            9. Combine sections and polish

            Use the tools available to accomplish these tasks in order.
            Always maintain context about workflow progress and data.
            Guide users through the workflow steps systematically.
            """,
            tools=[
                self.step1_scrape_headlines,
                self.step2_filter_ai_headlines,
                self.step3_fetch_article_texts,
                self.step4_summarize_articles,
                self.step5_extract_and_cluster_topics,
                self.step6_rate_articles,
                self.step7_organize_sections,
                self.step8_write_sections,
                self.step9_finalize_newsletter,
                self.get_workflow_status,
                self.run_complete_workflow,
                self.reset_workflow
            ]
        )

    @function_tool
    async def step1_scrape_headlines(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        sources: List[str] = None,
        max_articles_per_source: int = 50
    ) -> str:
        """Step 1: Scrape headlines and URLs from various sources"""
        if sources is None:
            sources = ["techcrunch", "arstechnica", "theverge", "wired", "venturebeat"]

        scraped_data = []

        # Mock scraping implementation (replace with real RSS/API scraping)
        for source in sources:
            for i in range(max_articles_per_source):
                article = {
                    'title': f"{source} AI Article {i+1}: Latest developments in machine learning",
                    'url': f"https://{source}.com/ai-article-{i+1}",
                    'source': source,
                    'published_at': (datetime.now() - timedelta(hours=i)).isoformat(),
                    'description': f"AI-related content from {source}"
                }
                scraped_data.append(article)

        wrapper.context.raw_headlines = scraped_data
        wrapper.context.scraped_urls = [article['url'] for article in scraped_data]
        wrapper.context.current_step = 1

        return f"✅ Step 1 Complete: Scraped {len(scraped_data)} headlines from {len(sources)} sources"


    @function_tool
    async def step2_filter_ai_content(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        ai_keywords: List[str] = None
    ) -> str:
        """Step 2: Filter headlines to AI-related content only"""
        if not wrapper.context.raw_headlines:
            return "❌ No headlines to filter. Run step 1 first."

        if ai_keywords is None:
            ai_keywords = [
                'ai', 'artificial intelligence', 'machine learning', 'deep learning',
                'neural network', 'llm', 'gpt', 'transformer', 'chatbot', 'automation',
                'computer vision', 'nlp', 'natural language', 'algorithm', 'model'
            ]

        ai_articles = []
        for article in wrapper.context.raw_headlines:
            title_lower = article['title'].lower()
            desc_lower = article['description'].lower()

            # Check if any AI keywords are present
            if any(keyword in title_lower or keyword in desc_lower for keyword in ai_keywords):
                ai_articles.append(article)

        wrapper.context.ai_headlines = pd.DataFrame(ai_articles)
        wrapper.context.current_step = 2

        return f"✅ Step 2 Complete: Filtered to {len(ai_articles)} AI-related headlines from {len(wrapper.context.raw_headlines)} total"

    @function_tool
    async def step3_fetch_article_texts(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Step 3: Fetch full article texts from URLs"""
        if wrapper.context.ai_headlines.empty:
            return "❌ No AI headlines to fetch. Complete steps 1-2 first."

        # Mock article fetching (replace with actual web scraping)
        article_texts = {}

        for _, row in wrapper.context.ai_headlines.iterrows():
            url = row['url']
            # Mock article content
            article_texts[url] = f"""
            {row['title']}

            This is a mock article about AI developments. In a real implementation,
            you would use libraries like requests + BeautifulSoup or newspaper3k
            to extract the full article text from the URL.

            Key points about this AI story:
            - Advancement in machine learning techniques
            - Impact on industry applications
            - Future implications for AI development

            This content would be much longer in practice, containing the full
            article text that needs to be summarized and analyzed.
            """

        wrapper.context.article_texts = article_texts
        wrapper.context.current_step = 3

        return f"✅ Step 3 Complete: Fetched full text for {len(article_texts)} articles"

    @function_tool
    async def step4_summarize_articles(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Step 4: Summarize each article to 3 key bullet points"""
        if not wrapper.context.article_texts:
            return "❌ No article texts to summarize. Complete steps 1-3 first."

        summaries = {}

        for url, text in wrapper.context.article_texts.items():
            # Mock summarization (replace with actual LLM summarization)
            summaries[url] = [
                "• Key development in AI technology or research",
                "• Practical implications for businesses or developers",
                "• Future outlook or next steps in this area"
            ]

        wrapper.context.article_summaries = summaries
        wrapper.context.current_step = 4

        return f"✅ Step 4 Complete: Generated 3-point summaries for {len(summaries)} articles"

    @function_tool
    async def step5_extract_and_cluster_topics(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        max_clusters: int = 8
    ) -> str:
        """Step 5: Extract topics and cluster articles"""
        if not wrapper.context.article_texts:
            return "❌ No articles to analyze. Complete steps 1-4 first."

        # Extract topics from each article (mock implementation)
        article_topics = {}
        all_topics = []

        for url, text in wrapper.context.article_texts.items():
            # Mock topic extraction (replace with NLP)
            topics = ['machine learning', 'business applications', 'research', 'ethics']
            article_topics[url] = topics
            all_topics.extend(topics)

        # Cluster articles by common topics
        topic_counts = Counter(all_topics)
        main_topics = [topic for topic, count in topic_counts.most_common(max_clusters)]

        topic_clusters = {}
        for topic in main_topics:
            topic_clusters[topic] = [
                url for url, topics in article_topics.items()
                if topic in topics
            ]

        wrapper.context.article_topics = article_topics
        wrapper.context.topic_clusters = topic_clusters
        wrapper.context.current_step = 5

        return f"✅ Step 5 Complete: Extracted topics and created {len(topic_clusters)} clusters"

    @function_tool
    async def step6_rate_articles(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        custom_rubric: Dict[str, str] = None
    ) -> str:
        """Step 6: Rate articles according to rubric"""
        if not wrapper.context.article_texts:
            return "❌ No articles to rate. Complete previous steps first."

        if custom_rubric:
            wrapper.context.rating_rubric.update(custom_rubric)

        # Mock rating (replace with actual evaluation)
        ratings = {}
        for url in wrapper.context.article_texts.keys():
            # Mock scoring based on rubric criteria
            relevance_score = 0.8
            novelty_score = 0.7
            impact_score = 0.9
            credibility_score = 0.8

            overall_rating = (relevance_score + novelty_score + impact_score + credibility_score) / 4
            ratings[url] = overall_rating

        wrapper.context.article_ratings = ratings
        wrapper.context.current_step = 6

        avg_rating = sum(ratings.values()) / len(ratings)
        return f"✅ Step 6 Complete: Rated {len(ratings)} articles. Average rating: {avg_rating:.2f}"

    @function_tool
    async def step7_organize_sections(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        target_sections: int = 10
    ) -> str:
        """Step 7: Organize articles into thematic sections"""
        if not wrapper.context.topic_clusters:
            return "❌ No topic clusters available. Complete steps 1-6 first."

        # Create thematic sections based on clusters and ratings
        sections = {}

        # Main thematic sections from top clusters
        top_clusters = sorted(
            wrapper.context.topic_clusters.items(),
            key=lambda x: len(x[1]),  # Sort by cluster size
            reverse=True
        )[:target_sections-1]  # Reserve space for "Other News"

        for topic, urls in top_clusters:
            # Only include high-rated articles
            high_rated_urls = [
                url for url in urls
                if wrapper.context.article_ratings.get(url, 0) >= 0.6
            ]
            if high_rated_urls:
                section_name = topic.title().replace('_', ' ')
                sections[section_name] = high_rated_urls

        # "Other News" section for remaining articles
        assigned_urls = set()
        for urls in sections.values():
            assigned_urls.update(urls)

        other_urls = [
            url for url in wrapper.context.article_texts.keys()
            if url not in assigned_urls and wrapper.context.article_ratings.get(url, 0) >= 0.5
        ]

        if other_urls:
            sections["Other News"] = other_urls

        wrapper.context.thematic_sections = sections
        wrapper.context.section_names = list(sections.keys())
        wrapper.context.current_step = 7

        section_summary = "\n".join([
            f"• {name}: {len(urls)} articles"
            for name, urls in sections.items()
        ])

        return f"✅ Step 7 Complete: Organized into {len(sections)} sections:\n{section_summary}"

    @function_tool
    async def step8_write_sections(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Step 8: Write content for each thematic section"""
        if not wrapper.context.thematic_sections:
            return "❌ No sections to write. Complete steps 1-7 first."

        section_drafts = {}

        for section_name, urls in wrapper.context.thematic_sections.items():
            # Gather content for this section
            section_articles = []

            for url in urls:
                summary = wrapper.context.article_summaries.get(url, [])
                rating = wrapper.context.article_ratings.get(url, 0)

                # Get article title from DataFrame
                article_row = wrapper.context.ai_headlines[
                    wrapper.context.ai_headlines['url'] == url
                ]
                title = article_row['title'].iloc[0] if not article_row.empty else "Unknown Title"

                section_articles.append({
                    'title': title,
                    'url': url,
                    'summary': summary,
                    'rating': rating
                })

            # Write section content (mock implementation)
            section_content = f"## {section_name}\n\n"

            for article in sorted(section_articles, key=lambda x: x['rating'], reverse=True):
                section_content += f"**{article['title']}**\n"
                for bullet in article['summary']:
                    section_content += f"{bullet}\n"
                section_content += f"[Read more]({article['url']})\n\n"

            section_drafts[section_name] = section_content

        wrapper.context.section_drafts = section_drafts
        wrapper.context.current_step = 8

        return f"✅ Step 8 Complete: Wrote content for {len(section_drafts)} sections"

    @function_tool
    async def step9_finalize_newsletter(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        newsletter_title: str = "AI Weekly Newsletter"
    ) -> str:
        """Step 9: Combine sections and polish final newsletter"""
        if not wrapper.context.section_drafts:
            return "❌ No section drafts available. Complete steps 1-8 first."

        # Combine all sections
        newsletter_content = f"# {newsletter_title}\n"
        newsletter_content += f"*Generated on {datetime.now().strftime('%B %d, %Y')}*\n\n"

        # Add introduction
        total_articles = len(wrapper.context.article_texts)
        newsletter_content += f"This week's AI newsletter covers {total_articles} key developments across {len(wrapper.context.section_drafts)} areas of AI.\n\n"

        # Add each section
        for section_name in wrapper.context.section_names:
            if section_name in wrapper.context.section_drafts:
                newsletter_content += wrapper.context.section_drafts[section_name]
                newsletter_content += "\n---\n\n"

        # Add footer
        newsletter_content += "*Thank you for reading! This newsletter was generated using AI curation and analysis.*"

        wrapper.context.final_newsletter = newsletter_content
        wrapper.context.workflow_complete = True
        wrapper.context.current_step = 9

        return f"✅ Step 9 Complete: Finalized newsletter with {len(wrapper.context.section_drafts)} sections"

    @function_tool
    async def get_workflow_status(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Get detailed workflow progress status"""
        state = wrapper.context

        status = {
            'current_step': state.current_step,
            'steps_completed': [
                f"1. Scraping: {len(state.raw_headlines)} headlines" if state.raw_headlines else "1. Scraping: Pending",
                f"2. AI Filtering: {len(state.ai_headlines)} AI articles" if not state.ai_headlines.empty else "2. AI Filtering: Pending",
                f"3. Text Fetching: {len(state.article_texts)} articles" if state.article_texts else "3. Text Fetching: Pending",
                f"4. Summarization: {len(state.article_summaries)} summaries" if state.article_summaries else "4. Summarization: Pending",
                f"5. Topic Clustering: {len(state.topic_clusters)} clusters" if state.topic_clusters else "5. Topic Clustering: Pending",
                f"6. Article Rating: {len(state.article_ratings)} rated" if state.article_ratings else "6. Article Rating: Pending",
                f"7. Section Organization: {len(state.thematic_sections)} sections" if state.thematic_sections else "7. Section Organization: Pending",
                f"8. Section Writing: {len(state.section_drafts)} drafts" if state.section_drafts else "8. Section Writing: Pending",
                f"9. Newsletter Finalization: {'Complete' if state.final_newsletter else 'Pending'}"
            ],
            'workflow_complete': state.workflow_complete
        }

        return f"Newsletter Workflow Status:\n\n" + "\n".join(status['steps_completed'])

    @function_tool
    async def run_complete_workflow(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        sources: List[str] = None,
        ai_keywords: List[str] = None
    ) -> str:
        """Run the complete 9-step workflow automatically"""
        results = []

        # Execute each step in sequence
        result1 = await self.step1_scrape_headlines(wrapper, sources)
        results.append(result1)

        result2 = await self.step2_filter_ai_content(wrapper, ai_keywords)
        results.append(result2)

        result3 = await self.step3_fetch_article_texts(wrapper)
        results.append(result3)

        result4 = await self.step4_summarize_articles(wrapper)
        results.append(result4)

        result5 = await self.step5_extract_and_cluster_topics(wrapper)
        results.append(result5)

        result6 = await self.step6_rate_articles(wrapper)
        results.append(result6)

        result7 = await self.step7_organize_sections(wrapper)
        results.append(result7)

        result8 = await self.step8_write_sections(wrapper)
        results.append(result8)

        result9 = await self.step9_finalize_newsletter(wrapper)
        results.append(result9)

        newsletter_length = len(wrapper.context.final_newsletter)

        return "\n".join(results) + f"\n\n🎉 Complete workflow finished! Newsletter ready ({newsletter_length} characters)"

    @function_tool
    async def reset_workflow(
        self,
        wrapper: RunContextWrapper[NewsletterState]
    ) -> str:
        """Reset workflow to start fresh"""
        wrapper.context.__dict__.update(NewsletterState().__dict__)
        return "🔄 Workflow reset. Ready to start step 1."

    @function_tool
    async def get_newsletter_preview(
        self,
        wrapper: RunContextWrapper[NewsletterState],
        max_chars: int = 500
    ) -> str:
        """Get a preview of the current newsletter"""
        if not wrapper.context.final_newsletter:
            return "Newsletter not ready yet. Complete the full workflow first."

        preview = wrapper.context.final_newsletter[:max_chars]
        if len(wrapper.context.final_newsletter) > max_chars:
            preview += "..."

        return f"Newsletter Preview:\n\n{preview}"

    async def run_step(self, user_input: str) -> str:
        """Run a workflow step with persistent state"""
        result = await Runner.run(
            self,
            user_input,
            session=self.session,
            context=self.state
        )
        return result.final_output

    def save_newsletter(self, filepath: str = None):
        """Save the final newsletter to file"""
        if not self.state.final_newsletter:
            print("No newsletter to save. Complete workflow first.")
            return

        if filepath is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filepath = f"ai_newsletter_{timestamp}.md"

        with open(filepath, 'w') as f:
            f.write(self.state.final_newsletter)

        print(f"Newsletter saved to {filepath}")




In [ ]:
import openai

client = openai.OpenAI(
  base_url="http://localhost:8787/v1",
  api_key=os.getenv("OPENAI_API_KEY"),
  default_headers={"x-portkey-provider": "openai"}
)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{"role": "user", "content": "Hello"}]
)
print(response.choices[0].message.content)

In [ ]:
from portkey_ai import Portkey

client = Portkey(
    provider="openai",
    Authorization=os.getenv("OPENAI_API_KEY")
)

# Example: Send a chat completion request
response = client.chat.completions.create(
    messages=[{"role": "user", "content": "Hello, how are you?"}],
    model="gpt-4o"
)

print(response.choices[0].message.content)

In [ ]:
type(prompt_template)

In [ ]:
class AgentState(TypedDict):
    """
    State of the LangGraph agent.
    Each node in the graph is a function that takes the current state and returns the updated state.
    """

    # the current working set of headlines (pandas dataframe not supported)
    AIdf: list[dict]
    # ignore stories before this date for deduplication (force reprocess since)
    model_low: str     # cheap fast model like gpt-4o-mini or flash
    model_medium: str  # medium model like gpt-4o or gemini-1.5-pro
    model_high: str    # slow expensive thinking model like o3-mini
    sources: dict  # sources to scrap
    sources_reverse: dict[str, str]  # map file names to sources

state = AgentState()


In [ ]:
SOURCES_FILE = "sources.yaml"

def initialize(state, sources_file=SOURCES_FILE) -> Dict[str, Any]:
    """Read and parse the sources.yaml file."""
    try:
        with open(sources_file, 'r', encoding='utf-8') as file:
            state["sources"] =  yaml.safe_load(file)
        state["sources_reverse"] = {v["title"]+".html":k for k,v in state["sources"].items()}
    except FileNotFoundError:
        raise FileNotFoundError(f"Sources file '{self.sources_file}' not found")
    except yaml.YAMLError as e:
        raise ValueError(f"Error parsing YAML file: {e}")

    return state


In [ ]:
state = initialize(state)
state
